In [ ]:
from azureml.core import Workspace, Experiment, Datastore, Environment, Dataset, Model, Run
from azureml.core.compute import ComputeTarget, AmlCompute, DataFactoryCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE, DEFAULT_GPU_IMAGE
from azureml.pipeline.core import Pipeline, PipelineParameter, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineParameter, PipelineData
from azureml.data.output_dataset_config import OutputTabularDatasetConfig, OutputDatasetConfig, OutputFileDatasetConfig
from azureml.data.datapath import DataPath
from azureml.data.data_reference import DataReference
from azureml.data.sql_data_reference import SqlDataReference
from azureml.pipeline.steps import DataTransferStep
from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

In [ ]:
ws = Workspace.from_config()

In [ ]:
from azureml.core.compute import ComputeTarget, AksCompute
from azureml.exceptions import ComputeTargetException

# Choose a name for your cluster
aks_name = "cluster-aks-gpu"

# Check to see if the cluster already exists
try:
    aks_target = ComputeTarget(workspace=ws, name=aks_name)
    print("Found existing compute target. So let's use it")
except ComputeTargetException:
    print('Creating a new compute target...')
    # Provision AKS cluster with GPU machine
    prov_config = AksCompute.provisioning_configuration(vm_size="STANDARD_NC6", 
                                                        location="eastus2")
    # Create the cluster
    aks_target = ComputeTarget.create(workspace=ws, 
                                      name=aks_name, 
                                      provisioning_configuration=prov_config)
    aks_target.wait_for_completion(show_output=True)

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AksWebservice, LocalWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

model_name = 'Street_Segmentation_Model'
deployment_name = 'street-segmentation-model'

model_list = Model.list(ws, model_name)
best_child_run_id = model_list[0].run_id
best_child_run = Run.get(ws, best_child_run_id)
model_list[0].download(exist_ok=True)
model = model_list[0]
environment = Environment.get(ws, 'AutoMLImages_ScoringEnv')
inference_config = InferenceConfig(entry_script='score.py', environment=environment, source_directory='./automl_outputs') # model_path = './automl_outputs/outputs/model.pt'


aks_config = AksWebservice.deploy_configuration(autoscale_enabled=True,                                                    
                                                cpu_cores=1,
                                                memory_gb=50,
                                                enable_app_insights=True)

aks_service = Model.deploy(ws,
                           models=[model],
                           inference_config=inference_config,
                           deployment_config=aks_config,
                           deployment_target=aks_target,
                           name=deployment_name,
                           overwrite=True)

aks_service.wait_for_deployment(show_output=True)
print()
print("Done. Model is deployed.")
print("\nAKS service status=", aks_service.state)

In [ ]:
from IPython.display import Image 
img = Image(filename='./sample_images/SSDB00038.JPG')
img.width = 750
display(img)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.lines import Line2D
from PIL import Image
import numpy as np
import json
import requests
import cv2

# for f in os.listdir('./test_images'):
#     if 'jpg' not in f:
#         continue
sample_image = './sample_images/SSDB00038.JPG'
data = open(sample_image, 'rb').read()
# Set the content type
headers = {'Content-Type': 'application/octet-stream', 'Authorization': f'Bearer {aks_service.get_keys()[0]}'}
scoring_uri = aks_service.scoring_uri

# Make the request and display the response
resp = requests.post(scoring_uri, data, headers=headers)

IMAGE_SIZE = (30,20)
plt.figure(figsize=IMAGE_SIZE)
img_np=mpimg.imread(sample_image)
img = Image.fromarray(img_np.astype('uint8'),'RGB')
x, y = img.size

fig,ax = plt.subplots(1, figsize=IMAGE_SIZE)#, figsize=(20,20)
# Display the image
ax.imshow(img_np)

covered_area  = 0.0
total_area  = x * y

covered_area_dict = {}
label_count_dict = {}
ax.set_axis_off()

# draw box and label for each detection 
detections = json.loads(resp.text)
for detect in detections['boxes']:
    label = detect['label']
    box = detect['box']
    polygon = detect['polygon']
    conf_score = detect['score']
    if label not in covered_area_dict.keys():
        covered_area_dict[label] = 0.0
        label_count_dict[label] = 0
    if conf_score > 0.6:
        ymin, xmin, ymax, xmax =  box['topY'],box['topX'], box['bottomY'],box['bottomX']
        topleft_x, topleft_y = x * xmin, y * ymin
        width, height = x * (xmax - xmin), y * (ymax - ymin)
        color = 'dodgerblue'
        if label == 'car':
            color = 'red'
        polygon_np = np.array(polygon[0])
        polygon_np = polygon_np.reshape(-1, 2)
        polygon_np[:, 0] *= x
        polygon_np[:, 1] *= y
        poly = patches.Polygon(polygon_np, True, facecolor=color, alpha=0.25)
        ax.add_patch(poly)
        poly_line = Line2D(polygon_np[:, 0], polygon_np[:, 1], linewidth=2,
                           marker='o', markersize=0, markerfacecolor=color, color=color)
        ax.add_line(poly_line)

        covered_area_dict[label] += cv2.contourArea(np.array(poly.xy).reshape((-1,1,2)).astype(np.int32))
        label_count_dict[label]+=1

plt.show()
display(img)
print()
print()